In [1]:
from numpy import mean
from numpy import std
from numpy import dstack
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import LSTM
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.regularizers import l2

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

4 Physical GPUs, 1 Logical GPU


In [3]:
s25 = np.load('s25.npy')
h25 = np.load('h25.npy')

In [4]:
s25_labels = np.array([0 for _ in range(0,len(s25))])
h25_labels = np.array([1 for _ in range(0,len(h25))])

In [5]:
X = np.append(s25,h25,axis=0)

In [6]:
y = np.append(s25_labels,h25_labels,axis=0)

In [7]:
X.shape

(1142, 6250, 19)

In [8]:
def evaluate_model(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 50, 128
    n_features = trainX.shape[2]
    n_steps, n_length = 10, 625
    trainX = trainX.reshape((trainX.shape[0], n_steps, n_length, n_features))
    testX = testX.reshape((testX.shape[0], n_steps, n_length, n_features))
    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)),
    input_shape=(None,n_length,n_features)))
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01))))
    model.add(TimeDistributed(Dropout(0.5)))
    model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(100, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01) ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy

In [9]:
def summarize_results(scores):
    print(scores)
    m, s = mean(scores), std(scores)
    print( ' Accuracy: %.3f%% (+/-%.3f) ' % (m, s))

In [10]:
def run_experiment(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    mean = trainX.mean(axis=0)
    trainX -= mean
    std = trainX.std(axis=0)
    trainX /= std
    testX -= mean
    testX /= std
    # repeat experiment
    scores = list()
    for r in range(repeats):
        score = evaluate_model(trainX, trainy, testX, testy)
        score = score * 100.0
        print( ' >#%d: %.3f ' % (r+1, score))
        scores.append(score)
    # summarize results
    summarize_results(scores)

In [11]:
run_experiment(X,y)

Epoch 1/50
8/8 [==============================] - 0s 51ms/step - loss: 10.3163 - accuracy: 0.5225
Epoch 2/50
8/8 [==============================] - 0s 49ms/step - loss: 6.0459 - accuracy: 0.5783
Epoch 3/50
8/8 [==============================] - 0s 49ms/step - loss: 4.4271 - accuracy: 0.6627
Epoch 4/50
8/8 [==============================] - 0s 48ms/step - loss: 3.7440 - accuracy: 0.7547
Epoch 5/50
8/8 [==============================] - 0s 49ms/step - loss: 3.2639 - accuracy: 0.7842
Epoch 6/50
8/8 [==============================] - 0s 50ms/step - loss: 2.8556 - accuracy: 0.8642
Epoch 7/50
8/8 [==============================] - 0s 49ms/step - loss: 2.5795 - accuracy: 0.8894
Epoch 8/50
8/8 [==============================] - 0s 50ms/step - loss: 2.3413 - accuracy: 0.9299
Epoch 9/50
8/8 [==============================] - 0s 50ms/step - loss: 2.2229 - accuracy: 0.9091
Epoch 10/50
8/8 [==============================] - 0s 51ms/step - loss: 2.0863 - accuracy: 0.9409
Epoch 11/50
8/8 [===========

8/8 [==============================] - 0s 50ms/step - loss: 0.8735 - accuracy: 0.9956
Epoch 35/50
8/8 [==============================] - 0s 50ms/step - loss: 0.8485 - accuracy: 0.9989
Epoch 36/50
8/8 [==============================] - 0s 50ms/step - loss: 0.8293 - accuracy: 0.9978
Epoch 37/50
8/8 [==============================] - 0s 49ms/step - loss: 0.8334 - accuracy: 0.9825
Epoch 38/50
8/8 [==============================] - 0s 50ms/step - loss: 0.8294 - accuracy: 0.9901
Epoch 39/50
8/8 [==============================] - 0s 49ms/step - loss: 0.8566 - accuracy: 0.9759
Epoch 40/50
8/8 [==============================] - 0s 50ms/step - loss: 0.8198 - accuracy: 0.9901
Epoch 41/50
8/8 [==============================] - 0s 51ms/step - loss: 0.9026 - accuracy: 0.9485
Epoch 42/50
8/8 [==============================] - 0s 50ms/step - loss: 0.8484 - accuracy: 0.9792
Epoch 43/50
8/8 [==============================] - 0s 51ms/step - loss: 0.7980 - accuracy: 0.9912
Epoch 44/50
8/8 [===============

8/8 [==============================] - 0s 48ms/step - loss: 1.5282 - accuracy: 0.9693
Epoch 18/50
8/8 [==============================] - 0s 49ms/step - loss: 1.4313 - accuracy: 0.9836
Epoch 19/50
8/8 [==============================] - 0s 50ms/step - loss: 1.3592 - accuracy: 0.9890
Epoch 20/50
8/8 [==============================] - 0s 50ms/step - loss: 1.3165 - accuracy: 0.9880
Epoch 21/50
8/8 [==============================] - 0s 51ms/step - loss: 1.2743 - accuracy: 0.9836
Epoch 22/50
8/8 [==============================] - 0s 51ms/step - loss: 1.2221 - accuracy: 0.9912
Epoch 23/50
8/8 [==============================] - 0s 51ms/step - loss: 1.1849 - accuracy: 0.9934
Epoch 24/50
8/8 [==============================] - 0s 49ms/step - loss: 1.1676 - accuracy: 0.9836
Epoch 25/50
8/8 [==============================] - 0s 49ms/step - loss: 1.2006 - accuracy: 0.9650
Epoch 26/50
8/8 [==============================] - 0s 49ms/step - loss: 1.1408 - accuracy: 0.9890
Epoch 27/50
8/8 [===============

8/8 [==============================] - 0s 51ms/step - loss: 0.7084 - accuracy: 0.9989
 >#5: 98.690 
[99.12663698196411, 99.56331849098206, 99.56331849098206, 99.12663698196411, 98.68995547294617]
 Accuracy: 99.214% (+/-0.327) 


In [12]:
def evaluate_model_2(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 50, 128
    n_features = trainX.shape[2]
    n_steps, n_length = 10, 625
    trainX = trainX.reshape((trainX.shape[0], n_steps, n_length, n_features))
    testX = testX.reshape((testX.shape[0], n_steps, n_length, n_features))
    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01),input_shape=(None,n_length,n_features))))
    model.add(LeakyReLU())
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01))))
    model.add(LeakyReLU())
    model.add(TimeDistributed(Dropout(0.5)))
    model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(100, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
    model.add(Dropout(0.5))
    model.add(Dense(100, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
    model.add(LeakyReLU())
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01) ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy

def run_experiment_2(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    mean = trainX.mean(axis=0)
    trainX -= mean
    std = trainX.std(axis=0)
    trainX /= std
    testX -= mean
    testX /= std
    # repeat experiment
    scores = list()
    for r in range(repeats):
        score = evaluate_model_2(trainX, trainy, testX, testy)
        score = score * 100.0
        print( ' >#%d: %.3f ' % (r+1, score))
        scores.append(score)
    # summarize results
    summarize_results(scores)
    
run_experiment_2(X,y)

Epoch 1/50
8/8 [==============================] - 0s 52ms/step - loss: 10.3067 - accuracy: 0.5356
Epoch 2/50
8/8 [==============================] - 0s 50ms/step - loss: 6.0398 - accuracy: 0.6484
Epoch 3/50
8/8 [==============================] - 0s 50ms/step - loss: 4.4227 - accuracy: 0.7306
Epoch 4/50
8/8 [==============================] - 0s 50ms/step - loss: 3.7701 - accuracy: 0.8182
Epoch 5/50
8/8 [==============================] - 0s 52ms/step - loss: 3.2783 - accuracy: 0.8916
Epoch 6/50
8/8 [==============================] - 0s 51ms/step - loss: 2.9156 - accuracy: 0.9299
Epoch 7/50
8/8 [==============================] - 0s 50ms/step - loss: 2.6802 - accuracy: 0.9332
Epoch 8/50
8/8 [==============================] - 0s 51ms/step - loss: 2.4882 - accuracy: 0.9485
Epoch 9/50
8/8 [==============================] - 0s 53ms/step - loss: 2.2965 - accuracy: 0.9671
Epoch 10/50
8/8 [==============================] - 0s 52ms/step - loss: 2.1883 - accuracy: 0.9595
Epoch 11/50
8/8 [===========

Epoch 27/50
8/8 [==============================] - 0s 50ms/step - loss: 1.0633 - accuracy: 0.9989
Epoch 28/50
8/8 [==============================] - 0s 50ms/step - loss: 1.0260 - accuracy: 0.9989
Epoch 29/50
8/8 [==============================] - 0s 50ms/step - loss: 0.9900 - accuracy: 0.9989
Epoch 30/50
8/8 [==============================] - 0s 48ms/step - loss: 0.9616 - accuracy: 0.9989
Epoch 31/50
8/8 [==============================] - 0s 50ms/step - loss: 1.4285 - accuracy: 0.8565
Epoch 32/50
8/8 [==============================] - 0s 51ms/step - loss: 1.3582 - accuracy: 0.9671
Epoch 33/50
8/8 [==============================] - 0s 51ms/step - loss: 1.1955 - accuracy: 0.9814
Epoch 34/50
8/8 [==============================] - 0s 52ms/step - loss: 1.0573 - accuracy: 0.9901
Epoch 35/50
8/8 [==============================] - 0s 53ms/step - loss: 0.9793 - accuracy: 0.9934
Epoch 36/50
8/8 [==============================] - 0s 50ms/step - loss: 0.9295 - accuracy: 0.9978
Epoch 37/50
8/8 [===

 >#3: 96.070 
Epoch 1/50
8/8 [==============================] - 0s 52ms/step - loss: 10.4163 - accuracy: 0.5542
Epoch 2/50
8/8 [==============================] - 0s 50ms/step - loss: 6.2227 - accuracy: 0.6616
Epoch 3/50
8/8 [==============================] - 0s 50ms/step - loss: 4.4583 - accuracy: 0.8007
Epoch 4/50
8/8 [==============================] - 0s 50ms/step - loss: 3.7185 - accuracy: 0.8817
Epoch 5/50
8/8 [==============================] - 0s 51ms/step - loss: 3.2191 - accuracy: 0.9200
Epoch 6/50
8/8 [==============================] - 0s 50ms/step - loss: 2.8845 - accuracy: 0.9430
Epoch 7/50
8/8 [==============================] - 0s 51ms/step - loss: 2.6444 - accuracy: 0.9496
Epoch 8/50
8/8 [==============================] - 0s 53ms/step - loss: 2.4278 - accuracy: 0.9770
Epoch 9/50
8/8 [==============================] - 0s 51ms/step - loss: 2.2982 - accuracy: 0.9650
Epoch 10/50
8/8 [==============================] - 0s 51ms/step - loss: 2.1736 - accuracy: 0.9660
Epoch 11/50
8/

8/8 [==============================] - 0s 50ms/step - loss: 1.2115 - accuracy: 0.9880
Epoch 27/50
8/8 [==============================] - 0s 51ms/step - loss: 1.1502 - accuracy: 0.9934
Epoch 28/50
8/8 [==============================] - 0s 49ms/step - loss: 1.1034 - accuracy: 0.9945
Epoch 29/50
8/8 [==============================] - 0s 51ms/step - loss: 1.0592 - accuracy: 0.9989
Epoch 30/50
8/8 [==============================] - 0s 50ms/step - loss: 1.0173 - accuracy: 1.0000
Epoch 31/50
8/8 [==============================] - 0s 50ms/step - loss: 1.0274 - accuracy: 0.9901
Epoch 32/50
8/8 [==============================] - 0s 50ms/step - loss: 1.1404 - accuracy: 0.9463
Epoch 33/50
8/8 [==============================] - 0s 49ms/step - loss: 1.0765 - accuracy: 0.9770
Epoch 34/50
8/8 [==============================] - 0s 50ms/step - loss: 1.0174 - accuracy: 0.9890
Epoch 35/50
8/8 [==============================] - 0s 50ms/step - loss: 0.9877 - accuracy: 0.9934
Epoch 36/50
8/8 [===============

In [13]:
def evaluate_model_3(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 50, 128
    n_features = trainX.shape[2]
    n_steps, n_length = 10, 625
    trainX = trainX.reshape((trainX.shape[0], n_steps, n_length, n_features))
    testX = testX.reshape((testX.shape[0], n_steps, n_length, n_features))
    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='selu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)),
    input_shape=(None,n_length,n_features)))
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='selu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01))))
    model.add(TimeDistributed(Dropout(0.5)))
    model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(100, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='selu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01) ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy

def run_experiment_3(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    mean = trainX.mean(axis=0)
    trainX -= mean
    std = trainX.std(axis=0)
    trainX /= std
    testX -= mean
    testX /= std
    # repeat experiment
    scores = list()
    for r in range(repeats):
        score = evaluate_model_3(trainX, trainy, testX, testy)
        score = score * 100.0
        print( ' >#%d: %.3f ' % (r+1, score))
        scores.append(score)
    # summarize results
    summarize_results(scores)
    
run_experiment_3(X,y)

Epoch 1/50
8/8 [==============================] - 0s 51ms/step - loss: 11.4587 - accuracy: 0.5597
Epoch 2/50
8/8 [==============================] - 0s 50ms/step - loss: 8.1510 - accuracy: 0.6955
Epoch 3/50
8/8 [==============================] - 0s 50ms/step - loss: 5.9099 - accuracy: 0.8018
Epoch 4/50
8/8 [==============================] - 0s 49ms/step - loss: 4.6438 - accuracy: 0.8806
Epoch 5/50
8/8 [==============================] - 0s 50ms/step - loss: 3.8713 - accuracy: 0.9507
Epoch 6/50
8/8 [==============================] - 0s 49ms/step - loss: 3.3869 - accuracy: 0.9704
Epoch 7/50
8/8 [==============================] - 0s 50ms/step - loss: 3.0959 - accuracy: 0.9671
Epoch 8/50
8/8 [==============================] - 0s 49ms/step - loss: 2.8405 - accuracy: 0.9901
Epoch 9/50
8/8 [==============================] - 0s 50ms/step - loss: 2.6135 - accuracy: 0.9901
Epoch 10/50
8/8 [==============================] - 0s 49ms/step - loss: 2.3900 - accuracy: 0.9945
Epoch 11/50
8/8 [===========

Epoch 27/50
8/8 [==============================] - 0s 50ms/step - loss: 1.4832 - accuracy: 0.9956
Epoch 28/50
8/8 [==============================] - 0s 50ms/step - loss: 1.3440 - accuracy: 0.9967
Epoch 29/50
8/8 [==============================] - 0s 50ms/step - loss: 1.2470 - accuracy: 1.0000
Epoch 30/50
8/8 [==============================] - 0s 50ms/step - loss: 1.1892 - accuracy: 1.0000
Epoch 31/50
8/8 [==============================] - 0s 50ms/step - loss: 1.1103 - accuracy: 0.9989
Epoch 32/50
8/8 [==============================] - 0s 50ms/step - loss: 1.0463 - accuracy: 1.0000
Epoch 33/50
8/8 [==============================] - 0s 50ms/step - loss: 1.0057 - accuracy: 0.9967
Epoch 34/50
8/8 [==============================] - 0s 51ms/step - loss: 0.9747 - accuracy: 0.9989
Epoch 35/50
8/8 [==============================] - 0s 50ms/step - loss: 0.9423 - accuracy: 0.9989
Epoch 36/50
8/8 [==============================] - 0s 50ms/step - loss: 0.9213 - accuracy: 0.9989
Epoch 37/50
8/8 [===

 >#3: 60.699 
Epoch 1/50
8/8 [==============================] - 0s 51ms/step - loss: 11.2641 - accuracy: 0.5926
Epoch 2/50
8/8 [==============================] - 0s 50ms/step - loss: 7.9046 - accuracy: 0.7755
Epoch 3/50
8/8 [==============================] - 0s 51ms/step - loss: 5.7870 - accuracy: 0.8620
Epoch 4/50
8/8 [==============================] - 0s 52ms/step - loss: 4.6368 - accuracy: 0.9321
Epoch 5/50
8/8 [==============================] - 0s 50ms/step - loss: 3.9666 - accuracy: 0.9441
Epoch 6/50
8/8 [==============================] - 0s 51ms/step - loss: 3.5176 - accuracy: 0.9584
Epoch 7/50
8/8 [==============================] - 0s 50ms/step - loss: 3.1395 - accuracy: 0.9825
Epoch 8/50
8/8 [==============================] - 0s 50ms/step - loss: 2.8936 - accuracy: 0.9847
Epoch 9/50
8/8 [==============================] - 0s 51ms/step - loss: 2.7473 - accuracy: 0.9869
Epoch 10/50
8/8 [==============================] - 0s 50ms/step - loss: 2.5625 - accuracy: 0.9836
Epoch 11/50
8/

8/8 [==============================] - 0s 50ms/step - loss: 1.2764 - accuracy: 0.9967
Epoch 27/50
8/8 [==============================] - 0s 51ms/step - loss: 1.2658 - accuracy: 0.9792
Epoch 28/50
8/8 [==============================] - 0s 50ms/step - loss: 1.3540 - accuracy: 0.9354
Epoch 29/50
8/8 [==============================] - 0s 50ms/step - loss: 1.5995 - accuracy: 0.9113
Epoch 30/50
8/8 [==============================] - 0s 50ms/step - loss: 1.7719 - accuracy: 0.9660
Epoch 31/50
8/8 [==============================] - 0s 51ms/step - loss: 1.7589 - accuracy: 0.9825
Epoch 32/50
8/8 [==============================] - 0s 50ms/step - loss: 1.5766 - accuracy: 0.9901
Epoch 33/50
8/8 [==============================] - 0s 49ms/step - loss: 1.4184 - accuracy: 0.9923
Epoch 34/50
8/8 [==============================] - 0s 50ms/step - loss: 1.2768 - accuracy: 0.9978
Epoch 35/50
8/8 [==============================] - 0s 52ms/step - loss: 1.1779 - accuracy: 0.9978
Epoch 36/50
8/8 [===============